<a href="https://colab.research.google.com/github/WeydisonAndrade/Sistema-de-Estoque.py/blob/main/Sistema_de_Estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Este Código importa e cria um banco de dados em SQLite


In [11]:

#Este código cria e importa o banco de dados
import sqlite3
from datetime import datetime

def criar_banco():
    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        senha TEXT,
        setor TEXT
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        quantidade INTEGER
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS movimentacoes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        produto_id INTEGER,
        tipo TEXT,
        setor TEXT,
        quantidade INTEGER,
        data_hora TEXT
    )
    """)

    conexao.commit()
    conexao.close()


In [2]:
# ESTE CÓDIGO CRIA A FUNCIONALIDADE DE LOGIN DE FUNCIONÁRIOS AUTORIZADOS
def login():
    nome = input("Usuário: ")
    senha = input("Senha: ")

    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("""
    SELECT * FROM usuarios
    WHERE nome=? AND senha=?
    """, (nome, senha))

    usuario = cursor.fetchone()
    conexao.close()

    if usuario:
        print("✅ Acesso autorizado!")
        return usuario
    else:
        print("❌ Acesso negado!")
        return None


In [3]:
# ESTE CÓDIGO IDENTIFICA OS PRODUTOS JÁ CADASTRADOS
def listar_produtos():
    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("SELECT * FROM produtos")
    produtos = cursor.fetchall()

    conexao.close()
    return produtos


In [4]:
# ESTE CÓDIGO REGISTRA A ENTRADA DE PRODUTOS NO SISTEMA DE ESTOQUE
def entrada_produto():
    produtos = listar_produtos()

    for p in produtos:
        print(f"{p[0]} - {p[1]} (Estoque: {p[2]})")

    produto_id = int(input("Selecione o ID do produto: "))
    quantidade = int(input("Quantidade recebida: "))
    data_hora = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("""
    UPDATE produtos
    SET quantidade = quantidade + ?
    WHERE id = ?
    """, (quantidade, produto_id))

    cursor.execute("""
    INSERT INTO movimentacoes
    (produto_id, tipo, setor, quantidade, data_hora)
    VALUES (?, 'ENTRADA', 'Fornecedor', ?, ?)
    """, (produto_id, quantidade, data_hora))

    conexao.commit()
    conexao.close()

    print("📦 Entrada registrada com sucesso!")


In [5]:
#REGISTRA A SAÍDA DE PRODUTOS DO SISTEMA POR SETOR
def saida_produto():
    produtos = listar_produtos()

    for p in produtos:
        print(f"{p[0]} - {p[1]} (Estoque: {p[2]})")

    produto_id = int(input("Selecione o ID do produto: "))
    setor = input("Setor solicitante: ")
    quantidade = int(input("Quantidade retirada: "))
    data_hora = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("""
    UPDATE produtos
    SET quantidade = quantidade - ?
    WHERE id = ? AND quantidade >= ?
    """, (quantidade, produto_id, quantidade))

    if cursor.rowcount == 0:
        print("❌ Estoque insuficiente!")
    else:
        cursor.execute("""
        INSERT INTO movimentacoes
        (produto_id, tipo, setor, quantidade, data_hora)
        VALUES (?, 'SAÍDA', ?, ?, ?)
        """, (produto_id, setor, quantidade, data_hora))

        conexao.commit()
        print("📤 Saída registrada com sucesso!")

    conexao.close()



In [6]:
# MENU PRINCIPAL DO SISTEMA
def menu():
    while True:
        print("""
        1 - Registrar entrada
        2 - Registrar saída
        3 - Listar produtos
        0 - Sair
        """)

        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            entrada_produto()
        elif opcao == "2":
            saida_produto()
        elif opcao == "3":
            for p in listar_produtos():
                print(p)
        elif opcao == "0":
            break
        else:
            print("Opção inválida!")


In [7]:
#EXECUÇÃO DO SISTEMA
criar_banco()

usuario = login()
if usuario:
    menu()


Usuário: Weydison Andrade
Senha: 527568
❌ Acesso negado!


In [8]:
#FUNÇÃO DE CADASTRAR USUÁRIO NO SISTEMA
import sqlite3

def cadastrar_usuario():
    nome = input("Nome do usuário: ")
    senha = input("Senha: ")
    setor = input("Setor (Almoxarifado/Logística): ")

    if setor.lower() not in ["almoxarifado", "logística", "logistica"]:
        print("❌ Setor não autorizado!")
        return

    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("""
        INSERT INTO usuarios (nome, senha, setor)
        VALUES (?, ?, ?)
    """, (nome, senha, setor))

    conexao.commit()
    conexao.close()

    print("✅ Usuário cadastrado com sucesso!")


In [9]:
#FUNÇÃO DE EXCLUIR USUÁRIO DO SISTEMA
def excluir_usuario():
    conexao = sqlite3.connect("estoque.db")
    cursor = conexao.cursor()

    cursor.execute("SELECT id, nome, setor FROM usuarios")
    usuarios = cursor.fetchall()

    if not usuarios:
        print("⚠️ Nenhum usuário cadastrado.")
        conexao.close()
        return

    print("\nUsuários cadastrados:")
    for u in usuarios:
        print(f"ID: {u[0]} | Nome: {u[1]} | Setor: {u[2]}")

    usuario_id = int(input("\nDigite o ID do usuário a ser excluído: "))

    cursor.execute("DELETE FROM usuarios WHERE id = ?", (usuario_id,))

    if cursor.rowcount == 0:
        print("❌ Usuário não encontrado.")
    else:
        conexao.commit()
        print("🗑️ Usuário excluído com sucesso!")

    conexao.close()
